In [3]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from sklearn.preprocessing import minmax_scale

In [4]:
PATH = './data_files/'

files = []
for f in os.listdir(PATH):
    if '.csv' in f:
        files.append(PATH + f)

#files = [PATH+f for f in os.listdir(PATH) if '.csv' in f]

In [5]:
def encode_timestamp(timestamp):
    """
    For encoding the 30 minute blocks into integers 
    """
    number_of_seconds = timestamp.hour * 3600 + timestamp.minute * 60
    return number_of_seconds // 1800 # 1800 because of 30-minute stepwidth


def transform_timestamp(df, col_name):
    """
    Transform timestamp to proper date/year/month/day values
    
    Args:
        df
        col_name: column of original dataframe based on which to infer dates
    """

    df['date'] = df[f'{col_name}'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S'))
    df['year'] = df[f'{col_name}'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S').year)
    df['month'] = df[f'{col_name}'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S').month)
    df['day'] = df[f'{col_name}'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S').day)

    """
    Robin: Encode the 30 minute intervals using integers
    """
    df['30min'] = df['date'].apply( encode_timestamp )
    
    return df

In [6]:
def numerical_to_float(df, cols):
    """
    Args:
        df:
        cols: columns to apply the dtype change to
    """
    for c in cols:
        try:
            df[f'{c}'] = df[f'{c}'].astype(dtype=float)
        except ValueError:
            # some files use ',' (comma) as decimal separator, replace with '.' (dot)
            df[f'{c}'] = df[f'{c}'].apply(lambda x: str(x).replace(',', '.'))
            df[f'{c}'] = df[f'{c}'].astype(dtype=float)
    
    return df

In [7]:
data = []

# numerical values to transform to float
cols = ['H_orig', 'LE_orig', 'ET_orig', 'CO2', 'H2O', 'NEE_orig', 'Reco', 'GPP_f', 'Ustar']
# unnecessary columns to be dropped
drop = ['TIMESTAMP_START', 'TIMESTAMP_MITTE', 'TIMESTAMP_ENDE', 'H_f', 'LE_f', 'ET_f', 'NEE_f']

for f in tqdm(files):
    try: 
        df = pd.read_csv(f, sep=',').drop(0)
    except pd.errors.ParserError: 
        df = pd.read_csv(f, sep=';').drop(0)

    # location based on file name (files should be properly labelled with either BG or GW!)
    # one-hot encode the location: BG (botanical garden)==0, GW (Goettinger forest)==1
    df['location'] = '0' if 'BG' in f else '1'

    df = transform_timestamp(df, 'TIMESTAMP_START')
    df = numerical_to_float(df, cols)
    df.drop(drop, axis=1, inplace=True)

    # drop any row containing NA values
    len_before = df.__len__()
    df.dropna(axis=0, how='any', inplace=True, ignore_index=True)
    na_removed = len_before - df.__len__()

    data.append(df)

100%|██████████| 4/4 [00:06<00:00,  1.66s/it]


In [8]:
# combine the preprocessed data into single dataframe
data_final = pd.concat(data, axis=0, ignore_index=True)

In [9]:
# rescaling all numerical values to be in range [0,1]
# alternatively center around 0 with unit std?
for col, type in zip(data_final.columns, data_final.dtypes):
    if type == 'float64':
        data_final[f'{col}'] = minmax_scale(data_final[f'{col}'])

data_final.head()

,H_orig,LE_orig,ET_orig,CO2,H2O,NEE_orig,Reco,GPP_f,Ustar,location,date,year,month,day,30min
0,0.311160,0.174929,0.172086,0.476698,0.242068,0.525569,0.048847,0.448053,0.195518,0,2023-02-16 12:30:00,2023,2,16,25
1,0.339654,0.192317,0.189252,0.474704,0.246260,0.533790,0.057604,0.441354,0.124494,0,2023-02-16 13:00:00,2023,2,16,26
2,0.342553,0.191697,0.188686,0.476755,0.244977,0.519500,0.062764,0.456326,0.165760,0,2023-02-16 13:30:00,2023,2,16,27
3,0.306098,0.177920,0.175130,0.476926,0.249284,0.530764,0.063945,0.445381,0.119010,0,2023-02-16 14:00:00,2023,2,16,28
4,0.296621,0.169818,0.167168,0.478749,0.257652,0.539033,0.066758,0.437665,0.126238,0,2023-02-16 14:30:00,2023,2,16,29


In [10]:
data_final.groupby('location').describe()

H_orig                                                             \
           count      mean       min       25%       50%       75%      max   
location                                                                      
0         4311.0  0.278935  0.089118  0.207594  0.263802  0.342388  0.72442   
1         5318.0  0.301783  0.000000  0.174758  0.226242  0.409975  1.00000   

                   LE_orig           ...   day             30min             \
               std   count     mean  ...   max       std   count       mean   
location                             ...                                      
0         0.088517  4311.0  0.26291  ...  31.0  8.970524  4311.0  24.429135   
1         0.172437  5318.0  0.22612  ...  31.0  8.772940  5318.0  23.366491   

                                                  
          min   25%   50%   75%   max        std  
location                                          
0         0.0  17.0  25.0  33.0  47.0  11.400534  
1         0.0  12.0  23.0  35.0  47.0  13.826858  

[2 rows x 112 columns]

In [11]:
data_final.to_csv('./data_preprocessed.csv')